# Python-1, Лекция 7

Лектор: Хайбулин Даниэль

Подготовил материал: Лущ Иван

Итак, сегодня мы поговорим про **исключения** и **контекстные менеджеры**.

### Исключения

Обработка ошибок — одна из наиболее дискуссионных тем в программировании. Можно сказать, что вокруг подходов к организации работы с ошибками во время выполнения программы формируются «религии» среди программистов.

Давайте рассмотрим ситуацию, в которой будет ошибка из-за попытки открытия несуществующего файла:

In [ ]:
with open("non_existent_file.txt", "r") as f:
    ...

В различных языках программирования и парадигмах вопросы обработки ошибок решаются по-разному. Наиболее распространённые подходы включают:

— использование кодов возврата (**return codes**) в языке `C`, когда функция сообщает об ошибке через специальное возвращаемое значение,

— введение специального типа (`Result` в `Rust`, `error` в `Go`, `std::expected` в `C++`) и сопутствующих конструкций для его обработки (например, `match` в `Rust`),

— применение механизма **исключений** (`exceptions`) в таких языках, как `Python`, `Java`, `C++`,

Подход к обработке ошибок оказывает существенное влияние и на стиль программирования (идиомы), и на удобство сопровождения программного кода, и на безопасность исполнения.

Рассмотрим дополнительные примеры возникновений исключений:

In [ ]:
1 / 0

In [ ]:
int("not a number")

In [ ]:
print(not_defined_variable)

In [ ]:
[0] * 1_000_000_000_000_000_000

В `Python` для перехвата исключений используется конструкция `try...except`:

In [ ]:
try:
    ... # код, который может вызвать исключение
except Exception:
    ... # обработка исключения

Перехват исключений позволяет:

- Не прерывать работу программы при возникновении ошибки.
- Корректно реагировать на разные типы ошибок.
- Выдавать понятные сообщения, делать повторные попытки и т.д.

К примеру:

In [ ]:
try:
    x = 1 / 0
except ZeroDivisionError:
    print("Нельзя делить на ноль!")

Можно обрабатывать несколько типов исключений:

In [ ]:
try:
    s = input("Введите число: ")
    n = int(s)
    print(10 / n)
except ValueError:
    print("Это не число!")
except ZeroDivisionError:
    print("Вы ввели ноль, деление невозможно.")

Можно перехватывать сразу несколько исключений, указав их в круглых скобках в виде кортежа (`tuple`) в блоке `except`:

In [ ]:
import typing as tp


def add_numbers(a: tp.Any, b: tp.Any) -> float:
    try:
        return float(a) + float(b)
    except (ValueError, TypeError) as err:
        print("Ошибка значения или типа:", err)
        return None


print(add_numbers("10", "5.5"))
print(add_numbers("abc", "3"))
print(add_numbers([1, 2], "3"))

Иногда бывает удобно перехватить сразу все «обычные» исключения (унаследованные от `Exception`), чтобы вывести ошибку пользователю (например, в пользовательском интерфейсе или логе), не прерывая всю программу аварийно:

In [ ]:
# вспомогательный код для следующей ячейки
with open("positive.txt", "w") as f:
    print(5, file=f)


with open("zero.txt", "w") as f:
    print(0, file=f)


with open("text.txt", "w") as f:
    print("text", file=f)

In [ ]:
def main() -> None:
    # Попробуйте ввести:
    # - positive.txt
    # - zero.txt
    # - text.txt
    # - non_existent_file.txt
    filename = input("Введите имя файла с числом: ")

    try:
        with open(filename) as f:
            num = int(f.read().strip())
        result = 100 / num
    except Exception as exc:
        print("Что-то пошло не так:", exc)
        return

    print("Результат деления 100 на", num, "равен", result)


if __name__ == "__main__":
    main()

В конструкции `try...except` можно также использовать дополнительные блоки `else` и `finally`, чтобы более гибко управлять поведением программы в зависимости от наличия или отсутствия ошибок.


- Блок `else` срабатывает только если в блоке `try` не возникло исключения.
- Блок `finally` выполняется в любом случае: была ошибка или нет (например, для освобождения ресурсов).


In [ ]:
# вспомогательный код для следующей ячейки
with open("data.txt", "w") as f:
    print("I hate exceptions", file=f)

In [ ]:
def main() -> None:
    # Попробуйте ввести:
    # - data.txt
    # - non_existent_file.txt
    filename = input()

    try:
        file = open(filename)
    except FileNotFoundError:
        print("Файл не найден.")
    else:
        print("Файл успешно открыт!")
        # Здесь можно безопасно работать с файлом, если открытие прошло без ошибок
        print("Первые 10 символов файла:", file.read(10))
    finally:
        # Этот блок выполнится всегда
        try:
            file.close()
            print("Файл закрыт.")
        except NameError:
            print("Файл не был открыт, закрывать нечего.")


if __name__ == "__main__":
    main()

В стандартной библиотеке Python определено множество видов исключений, структурированных в виде иерархии. Ниже приведена полная иерархия стандартных исключений:

https://docs.python.org/3/library/exceptions.html#exception-hierarchy

```
BaseException
 ├── BaseExceptionGroup
 ├── GeneratorExit
 ├── KeyboardInterrupt
 ├── SystemExit
 └── Exception
      ├── ArithmeticError
      │    ├── FloatingPointError
      │    ├── OverflowError
      │    └── ZeroDivisionError
      ├── AssertionError
      ├── AttributeError
      ├── BufferError
      ├── EOFError
      ├── ExceptionGroup [BaseExceptionGroup]
      ├── ImportError
      │    └── ModuleNotFoundError
      ├── LookupError
      │    ├── IndexError
      │    └── KeyError
      ├── MemoryError
      ├── NameError
      │    └── UnboundLocalError
      ├── OSError
      │    ├── BlockingIOError
      │    ├── ChildProcessError
      │    ├── ConnectionError
      │    │    ├── BrokenPipeError
      │    │    ├── ConnectionAbortedError
      │    │    ├── ConnectionRefusedError
      │    │    └── ConnectionResetError
      │    ├── FileExistsError
      │    ├── FileNotFoundError
      │    ├── InterruptedError
      │    ├── IsADirectoryError
      │    ├── NotADirectoryError
      │    ├── PermissionError
      │    ├── ProcessLookupError
      │    └── TimeoutError
      ├── ReferenceError
      ├── RuntimeError
      │    ├── NotImplementedError
      │    ├── PythonFinalizationError
      │    └── RecursionError
      ├── StopAsyncIteration
      ├── StopIteration
      ├── SyntaxError
      │    └── IndentationError
      │         └── TabError
      ├── SystemError
      ├── TypeError
      ├── ValueError
      │    └── UnicodeError
      │         ├── UnicodeDecodeError
      │         ├── UnicodeEncodeError
      │         └── UnicodeTranslateError
      └── Warning
           ├── BytesWarning
           ├── DeprecationWarning
           ├── EncodingWarning
           ├── FutureWarning
           ├── ImportWarning
           ├── PendingDeprecationWarning
           ├── ResourceWarning
           ├── RuntimeWarning
           ├── SyntaxWarning
           ├── UnicodeWarning
           └── UserWarning
```


`BaseException` — "корень" всей иерархии; все остальные исключения наследуются от этого класса.
Исключения делятся на две больших ветви:

- Исключения, наследуемые от `BaseException` (**но НЕ от `Exception`**), например:
    - `KeyboardInterrupt` — прерывание с клавиатуры (`Ctrl+C`)
    
    - `SystemExit` — выход из интерпретатора
    
    - `GeneratorExit` — завершение генератора (узнаем чуть позже, что такое генераторы и что это за исключение)
    
    Эти обычно не стоит перехватывать в `except Exception`, иначе вы блокируете возможность корректного выхода из программы.

- Все остальные (от `Exception`) — это исключения, появляющиеся в повседневном коде, которые обычно мы и обрабатываем.


`try...except` без уточнения типа **перехватывает все исключения**, то есть любые экземпляры, наследуемые от `BaseException`, включая `SystemExit`, `KeyboardInterrupt`, `GeneratorExit` и даже пользовательские, которые напрямую наследуются от `BaseException`.

In [ ]:
try:
    raise KeyboardInterrupt("CTRL+C")
except:
    print("Перехвачено!")

Уже отмечалось, что перехват исключений, наследуемых непосредственно от `BaseException`, но не являющихся подклассами `Exception`, (таких как `SystemExit`, `KeyboardInterrupt`, `GeneratorExit` и др.) считается плохой практикой и, как правило, должен избегаться. 

Для отлова обычных ошибок рекомендуется использовать конструкцию `except Exception:`, чтобы не нарушать корректную работу интерпретатора `Python`, такую как, например, возможность завершить программу с помощью сочетания клавиш `Ctrl+C (KeyboardInterrupt)`.

Важные группы `Exception`:


- `ArithmeticError`: ошибки арифметики — деление на ноль, переполнение, ошибки с плавающей точкой.
- `LookupError`: для неудачных обращений по индексу (`IndexError`) и ключу (`KeyError`).
- `OSError`: все ошибки, связанные с операционной системой — файлы, процессы, соединения.
- `RuntimeError`, `ValueError`, `TypeError` — "общие" ошибки, возникающие при неправильных входных данных, нарушениях логики и т.п.
- `ImportError`, `ModuleNotFoundError`: ошибки импорта модулей.
- `SyntaxError`, `IndentationError`, `TabError`: нарушения синтаксиса, проблемы с отступами в исходном коде.


![](1f11cc1d97095debe72a443bc576e2ed.jpg)

`IndentationError` — это разновидность `SyntaxError` и относится к исключениям уровня компиляции, возникает во время разбора (парсинга) кода `Python`, до его фактического выполнения. Поэтому перехватить `IndentationError` в обычном `try...except` вокруг кода с ошибкой невозможно — такой код даже не исполнится.

In [ ]:
def test():
print("Ошибка отступа!")

Однако можно обработать ситуацию, когда вы исполняете или компилируете строку с ошибочной программой во время выполнения. Например, с помощью функции `exec()`:

In [ ]:
code = """
def test():
print("Ошибка отступа!")
"""

try:
    # функция exec выполняет код, представленный в виде строки
    exec(code)
except IndentationError as e:
    print("IndentationError:", e)
except SyntaxError as e:
    print("Другая синтаксическая ошибка:", e)

В представленной иерархии существует, пожалуй, наиболее необычный (на взгляд автора) подкласс исключений — `Warning`.

`Warning` и подклассы используются не для ошибок, а для сообщений-предупреждений (не прерывают выполнение, можно перехватывать через модуль `warnings`). Подробное рассмотрение предупреждений будет проведено в заключительной части данного топика.



Отдельно следует отметить, что начиная с `Python 3.11` появились **exception-группы** (`ExceptionGroup`), предназначенные для поддержки работы с несколькими одновременно возникшими исключениями, что особенно актуально при работе с асинхронным кодом. Для обработки таких групп введена специальная конструкция `except*`. Однако в рамках данного материала мы не будем подробно рассматривать этот механизм, поскольку для его понимания требуется предварительное освоение принципов работы с корутинами.



Теперь, когда мы разобрались с перехватом исключений, рассмотрим, как инициировать их возникновение самостоятельно (так называемое "бросание" исключений):

In [ ]:
raise Exception("Исключение!")

Оператор `raise` требует, чтобы в качестве аргумента ему передавался объект, являющийся экземпляром класса, производного от `BaseException`. Передача в качестве аргумента объекта, не являющегося экземпляром класса, производного от `BaseException` (например, строки или числа), приведёт к возникновению исключения `TypeError`:

In [ ]:
raise 1

In [ ]:
# здесь все ок
a = ValueError("value error")

raise a

А ещё можно использовать `raise` внутри блока `except` без указания типа исключения — это позволяет повторно пробросить текущее исключение после выполнения необходимой дополнительной обработки, например, логирования или очистки ресурсов:

In [ ]:
try:
    1 / 0
except ZeroDivisionError:
    print("Логируем ошибку и пробрасываем дальше")
    raise

Ещё одной важной возможностью является использование конструкции `raise ... from ...` С её помощью можно явно указать причинно-следственную связь между двумя исключениями: если одно исключение возникает в процессе обработки другого, новое исключение может быть выброшено с ссылкой на исходное. Такой приём называется **цепочкой исключений** (`exception chaining`) и облегчает отладку и анализ ошибок, позволяя видеть, какое исключение послужило причиной нового.

In [ ]:
try:
    int("abc")
except ValueError as e:
    raise RuntimeError("Ошибка преобразования данных") from e

В некоторых случаях стандартных исключений Python бывает недостаточно для выражения специфических ошибок, характерных именно для вашей программы или библиотеки. В таких ситуациях рекомендуется определять собственные типы исключений:

In [ ]:
class MyCustomError(Exception):
    """Описание пользовательской ошибки"""
    ...

# Использование собственного исключения:
raise MyCustomError("Что-то пошло не так")

Рекомендуется давать пользовательским классам исключений имена, оканчивающиеся на `Error` ([pep8](https://peps.python.org/pep-0008/#exception-names)).

In [ ]:
class MySubCustomError(MyCustomError):
    def __str__(self):
        # self.args - это аргументы, которые мы передали в __init__
        return f"Some information of my sub custom error: {self.args}"


try:
    raise MySubCustomError(1)
except MyCustomError as e:
    print(f"Что-то не так: {e=}")
    raise

В `Python` также существует специализированный модуль `traceback`, предназначенный для получения и обработки текстового представления трассировки стека (`stack trace`). Подробное знакомство с возможностями этого модуля предлагается провести в рамках практических занятий.


Также на семинарских занятиях целесообразно более подробно рассмотреть использование специальных атрибутов исключений: `__cause__`, `__context__`, `__traceback__`.

Остаётся рассмотреть вопросы, связанные с обработкой предупреждений (`warning`) в Python:

Повторимся. В `Python` предупреждения (`Warning`) — это не исключения, они не останавливают программу, но позволяют информировать программиста или пользователя о потенциальных проблемах. Используются через стандартный модуль `warnings`.

In [ ]:
import warnings

warnings.warn("Это предупреждение!")

Примеры разных типов `warning`:

`UserWarning` — базовое, "пользовательское" предупреждение (по умолчанию):

In [ ]:
warnings.warn("Обычное пользовательское предупреждение!")

`DeprecationWarning` — предупреждение об устаревшем функционале:

In [ ]:
warnings.warn("Этот метод устарел и будет удалён.", DeprecationWarning)

`SyntaxWarning` — замечание по поводу опасного синтаксиса (чаще всего выводится самим интерпретатором):

In [ ]:
x = "hello"
if x is "hello":
    print("Строки совпали")

`BytesWarning` — предупреждение работы со строками и байтами

In [ ]:
a = "text"
b = b"text"

print(a == b)  # В обычном режиме — False, но если запустить с python -b, будет BytesWarning
# -b     : issue warnings about converting bytes/bytearray to str and comparing
#          bytes/bytearray with str or bytes with int. (-bb: issue errors)

`RuntimeWarning` — предупреждение, связанное с логикой выполнения

`FutureWarning` — то, что будет изменено в будущем

`ResourceWarning` — предупреждения об утечках ресурсов

Модуль `warnings` не вызывает исключение, но можно изменить поведение, чтобы, например, превратить предупреждение в ошибку:

In [ ]:
warnings.simplefilter("error")
try:
    warnings.warn("Это уже ошибка!", UserWarning)
except UserWarning as e:
    print("Поймали как ошибку:", e)

Возвращаем режим по умолчанию:

In [ ]:
warnings.simplefilter("default")

Можно создать свой подкласс `warning`:

In [ ]:
class MyWarning(Warning):
    pass

warnings.warn("Специальное предупреждение!", MyWarning)


Предупреждения (`warning`) в `Python` могут быть программно подавлены или отключены.
На практике это часто делается при помощи модуля `warnings`, чтобы игнорировать нежелательные предупреждения в ходе выполнения кода. Например, при выполнении лабораторных работ по линейной алгебре или геометрии в начале программы вы возможно встретите следующую строку:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
warnings.warn("Это предупреждение!")

In [ ]:
warnings.simplefilter("default")

Игнорировать предупреждения (`warnings`) —  плохая практика. Лучше разбираться в их причинах и фиксить, либо подавлять выборочно осознанно.

Игнорировать только определенный класс предупреждений:

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

С определенным текстом:

In [ ]:
import warnings
warnings.filterwarnings("ignore", message=".*устарел.*")

Только в определенном модуле:

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="my_module")

Игнорировать только внутри блока (самый лучший подход, если уж нужно заигнорировать):

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    warnings.warn("Это пользовательское предупреждение!", UserWarning)  # Не будет отображено

warnings.warn("Это пользовательское предупреждение!", UserWarning)  # Будет отображено

### Контекстные менеджеры